In [ ]:
# !ls ${CUDA_DIR}/nvvm/libdevice
# !set MLIR_CRASH_REPRODUCER_DIRECTORY="enable"
# !set LD_LIBRARY_PATH="~/.local/lib/python3.10/site-packages/tensorrt"
# !ls ~/.local/lib/python3.10/site-packages/tensorrt

In [ ]:
import os
# os.environ["OMP_NUM_THREADS"] = "12"
# os.environ["LD_LIBRARY_PATH"]="~/.local/lib/python3.10/site-packages/tensorflow"
# os.environ["LD_LIBRARY_PATH"]="/home/pedro/anaconda3/envs/tensorflow/lib"

In [ ]:
# nvidia_base = "~/.local/lib/python3.10.site-packages/nvidia"

# nvidia_paths = ["cusparse", "cusolver", "cuda_runtime"]

# nvidia_libs = []
# for np in nvidia_paths:
#     nvidia_libs.append(f"{nvidia_base}/{np}/lib")


# cuda = ["/usr/lib/x86_64-linux-gnu","/usr/local/cuda-12.1/lib64", "~/.local/lib/python3.10/site-packages/tensorrt", *nvidia_libs]

# ld_library_path = ":".join(cuda)

# os.environ["LD_LIBRARY_PATH"] = ld_library_path
# ld_library_path

In [ ]:
%load_ext filprofiler

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !wget https://drive.google.com/uc?export=download&id=0B7EVK8r0v71pZjFTYXZWM3FlRnM

In [ ]:
# !pip install ipywidgets

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import math
from json import dumps
from tqdm.notebook import tqdm, trange
from IPython import display
import tensorflow_addons as tfa
# for g in gpus:
#     tf.config.experimental.set_memory_growth(g, True)
    
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout

from tensorflow.keras.applications import ResNet50V2 as ResNet
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Rescaling
from tensorflow.keras.regularizers import L2
from enum import Enum

from tensorflow.keras.models import Model, Sequential
from matplotlib import pyplot as plt
from IPython import display # If using IPython, Colab or Jupyter
import numpy as np
import tensorflow_addons as tfa
import datetime
import random
from sklearn.model_selection import train_test_split
import time

import os
import re
import pathlib

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
print(f"gpus={gpus}")

tf.config.threading.set_inter_op_parallelism_threads(8) 
tf.config.threading.set_intra_op_parallelism_threads(8)
# if gpus:
#     try:
#         total_gpu_memory = 23817
#         tf.config.experimental.set_virtual_device_configuration(gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=total_gpu_memory-1000)])
#     except RuntimeError as e:
#         print(e)

In [ ]:
print(tf.__version__)

In [ ]:
from utils import TensorboardUtil

In [ ]:
import tensorflow_datasets as tfds
from functools import reduce
splits = tfds.even_splits('train', n=200, drop_remainder=True)

from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

## Dataset

In [ ]:
import datetime

def now():
    now = datetime.datetime.now()
    return now.strftime('%Y_%m_%d_T%H_%M_%S') + ('_%02d' % (now.microsecond / 10000))

def minute():
    now = datetime.datetime.now()
    return now.strftime('%H_%M') 

minute()

## Perceptual loss

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow.keras.backend as K



class LossNetwork(tf.keras.models.Model):
    def __init__(self):
        super(LossNetwork, self).__init__()
        vgg = VGG16(include_top=False, weights='imagenet')
        vgg.trainable = False
        
        self.selected_layers = ['block1_conv1', 'block2_conv2',"block3_conv3" ,'block4_conv3','block5_conv3']
        self.selected_layer_weights = [0.1, 0.4 , 0.4 , 0.8 , 1.6]
        
        model_outputs = [vgg.get_layer(name).output for name in self.selected_layers]
        self.model = tf.keras.models.Model(vgg.input, model_outputs)
        # mixed precision float32 output
        self.linear = layers.Activation('linear', dtype='float32') 

    def call(self, x):
        x = tf.cast(x, tf.float32)
        x = preprocess_input(x)
        x = self.model(x)
        x = self.linear(x)
        return x

#     @tf.function
    def loss(self, x, y):
        h1_list = self.model(x)
        h2_list = self.model(y)
        
        rc_loss = 0.0
        
        for h1, h2, weight in zip(h1_list, h2_list, self.selected_layer_weights):
            h1 = tf.cast(h1, tf.float32)
            h2 = tf.cast(h2, tf.float32)
            h1 = K.batch_flatten(h1)
            h2 = K.batch_flatten(h2)
            rc_loss = rc_loss + tf.cast(tf.reduce_mean((h1 - h2)**2), tf.float32) * weight   
            rc_loss = tf.cast(rc_loss, tf.float32)
      
        return rc_loss
    

    def loss2(self, x, y):
        h1_list = self.model(x)
        h2_list = self.model(y)
        
        rc_loss = 0.0
        
        for h1, h2, weight in zip(h1_list, h2_list, self.selected_layer_weights):
            h1 = tf.cast(h1, tf.float32)
            h2 = tf.cast(h2, tf.float32)

            h1 = h1 / 100
            h2 = h2 / 100
#             h1 = K.batch_flatten(h1)
#             h2 = K.batch_flatten(h2)
            rc_loss = rc_loss + tf.cast(tf.reduce_mean((h1 - h2)**2, axis=[1,2,3]), tf.float32) * weight   
            rc_loss = tf.cast(rc_loss, tf.float32)
      
        return rc_loss
    
class PeceptualLoss(tf.keras.losses.Loss):
    def __init__(self):
        super(PeceptualLoss, self).__init__()
        self.loss_network = LossNetwork()
        self.loss_network.trainable = False
        
    def call(self, y_true, y_pred):
        return self.loss_network.loss(y_true,y_pred)

    
def build_perceptual_loss(*args, **kwargs):
    return PeceptualLoss()


## Render test images

In [ ]:
from utils import ImageRenderer

## Gradient accumulator

In [ ]:
from utils import CustomTrainStep

Best result yet:

- In 150/300 epochs, with ranges best_lr_range = [3e-5, 5e-7] and linspace of size 300.

## Trainer tool

## Training

In [ ]:
def save_weights(layer_number, name):
    def fn(model):
        model.layers[layer_number].save_weights(name)
        print(f"Weights for layer {model.layers[layer_number].name} saved at {name}")
    return fn

def save_layers(layer_names, names):
    def fn(model):
        for layer, name in zip(list(filter(lambda layer: layer.name in layer_names, model.layers)), names):
            layer.save_weights(name)
            print(f"Weights for layer {layer.name} saved at {name}")
    return fn

def save_weights_from_map(name_map):
    def fn(model):
        print("Saving weights")
        model_map = extract_models(model)
        for key, value in model_map.items():
            print(f"Saving {value.name} to {name_map[key]}")
            value.save_weights(name_map[key])
        print(f"Saved weights")
        
    return fn

def save_multiple_weights(layer_numbers_with_names):
    def fn(model):
        for layer_number, name in layer_numbers:
            model.layers[layer_number].save_weights(name)
            print(f"Weights for layer {model.layers[layer_number].name} saved at {name}")
    return fn

# save_layers(["decoder_small"], ['./weights/decoder_small'])(smodel)

- ELU https://deeplearninguniversity.com/elu-as-an-activation-function-in-neural-networks/: Allowed to get more details through the network without connecting layers from the decoder to layers in the decoder

In [ ]:


def resize_input(inputs, output_shape=(120, 176), rescale_input=True):
    oh = output_shape[0]
    ow = output_shape[1]
    y = tf.keras.layers.Rescaling(1./255)(inputs) if rescale_input else inputs
    y = tf.cast(y, tf.float16)
    y = tf.image.resize(
        y,
        [oh,ow],
        preserve_aspect_ratio=False,
        antialias=False,
        name=None)
    return y

class DecoderSize(Enum):
    xs = (60, 90)
    sm = (240, 360)
    md = (480, 720)
    lg = (720, 1080)
    
class CustomUpSampling2D(tf.keras.layers.Layer):
  def __init__(self, size):
    super(CustomUpSampling2D, self).__init__()
    if type(size) is not tuple and type(size) is not list:
        size = (size, size)
    self.size = size

  def build(self, input_shape):
    pass

  def call(self, input):
      return tf.repeat(tf.repeat(input, self.size[0], axis=1), self.size[1], axis=2)
        
class LayerFactory:
    def __init__(self):
        self.downs = 0
        self.ups = 0
        self.resbs = 0
        
    def get_up_name(self):
        self.ups += 1
        return self.ups
    
    def get_down_name(self):
        self.downs += 1
        return self.downs
    
    def get_resb_name(self, name=None):
        if name is None:
            self.resbs+=1
            return self.resbs
        return name
        
        
    def conv(self, filters, kernel_size=3, strides=1, activation="elu", norm=True, **args):
        def fn(x):
            x = layers.Conv2D(filters,
                             kernel_size=kernel_size,
                             strides=strides,
                             padding="same",
                             use_bias=True,
#                              activation=activation,
#                              kernel_regularizer=L2(1e-2),
                             **args)(x)
#             if norm:
#                 x = layers.BatchNormalization()(x)
            return layers.Activation(activation)(x)
        return fn
    
    def down(self, filters, kernel_size=3,strides=2):
        def fn(x):
            x = self.conv(filters, kernel_size=kernel_size, strides=2, norm=True)(x)
            return self.resblock(filters, kernel_size, name=f"fdown_{self.get_down_name()}", norm=True)(x)
        return fn
        
    def convblock(self, filters, kernel_size=3, name=None):
        def fn(x, ff=None):
            return self.resblock(filters, kernel_size)(x)
        return fn

    def resblock(self, filters, kernel_size=3, activation="elu", name=None, norm=True):
        def fn(x):
            resbname = self.get_resb_name(name)
            x = self.conv(filters, activation=activation, norm=norm, name=f"{resbname}_resb_conv_1")(x)
            y = self.conv(filters, activation=activation, norm=norm, name=f"{resbname}_resb_conv_2")(x)
            y = self.conv(filters, activation=activation, norm=norm, name=f"{resbname}_resb_conv_3")(y)
            out = layers.Add()([x,y]) / 2
#             out = layers.BatchNormalization()(out)
            return tf.keras.activations.elu(out)
        return fn
    
    
    def up(self, filters, kernel_size=3):
        def fn(x):
#             x = tf.stop_gradient(tf.keras.layers.UpSampling2D(size=(2, 2))(x))
#             x = CustomUpSampling2D(size=(2, 2))(x)
            x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)

#             x = upsample_no_grads(x)
#             x = self.conv(filters, kernel_size=kernel_size, strides=1, norm=False)(x)
            x = self.resblock(filters, kernel_size, name=f"up_{self.get_up_name()}", norm=False)(x)
            return x
        return fn
    

class MyRescale(tf.keras.layers.Layer):
    def __init__(self):
        super(MyRescale, self).__init__()

    def call(self, inputs):
        mean = 255./2
        dev = 255
        return  (inputs - mean) / dev
    
    
def build_small_decoder(decoder_size=DecoderSize.xs):
    l = LayerFactory()
    
    inputs0 = keras.Input(shape=(480, 720, 3), dtype=tf.float16, name="autoencoder_input")
    i1 = MyRescale()(inputs0)

    s1 = l.resblock(64,kernel_size=3, norm=False)(i1)
    s2 = l.resblock(64,kernel_size=5, norm=False)(i1)
    
    x = tf.add(s1,s2) / 2
#     x = layers.BatchNormalization()(x)
    x = tf.keras.activations.elu(x)
   
    x = l.down(128)(x)
    x = l.down(256)(x)
#     x = layers.SpatialDropout2D(0.1)(x)
    x = l.down(512)(x)
    x = layers.SpatialDropout2D(0.1)(x)
    x = l.down(512)(x)
    x = layers.SpatialDropout2D(0.1)(x)

    x = l.up(512)(x)    
    x = l.up(512)(x)
    x = l.up(128)(x)
    x = l.up(64)(x)
    x = l.resblock(32 ,3, norm=False)(x)
    x = l.resblock(9 ,3, norm=False)(x)
    x = l.conv(3, 3, activation="sigmoid", norm=False)(x)
    x = tf.keras.layers.Rescaling(255.)(x)
    x = layers.Activation("linear", dtype=tf.float32)(x)

    return tf.keras.Model(inputs0, x, name="decoder_small")



model = build_small_decoder()
model.summary()


In [ ]:
import sys
from keras import backend as K
model.layers[2].weights, sys.getsizeof(model.layers[2].weights)

# [K.batch_flatten(l) for l in model.layers[2:]]
mae = tf.keras.losses.MeanAbsoluteError()

def compare_model_weights(m1, m2):
    ws = []
    for i, l1 in enumerate(m1.layers):
        l2 = m2.layers[i]
        if len(l1.weights) > 0:
            for wi, w1 in enumerate(l1.weights):
                w1 = K.batch_flatten(l1.weights[wi])
                w2 = K.batch_flatten(l2.weights[wi])
#             print(l1.weights[2].shape, l2.weights[2].shape)
                ws.append(mae(w1, w2))
    return tf.math.reduce_sum(ws)

# model.load_weights("weights/denoiser_good")
# mse(get_model_weights(build_small_decoder()), get_model_weights(model))


class GoodModelMetric(tf.keras.metrics.Metric):
    def __init__(self, other_model, **kwargs):
        super(GoodModelMetric, self).__init__(name="GoodModelMetric", **kwargs)
        
        self.good_model = build_small_decoder()
        self.good_model.load_weights("weights/denoiser_good")
        self.sim = self.add_weight(name='similarity', initializer='zeros')
        self.other_model = other_model

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.sim.assign(compare_model_weights(self.good_model, self.other_model))

    def result(self):
        return self.sim

# wm1 = compare_model_weights(model, build_small_decoder())
# # np.pad(wm1, pad_width=((0, 0), (1, 1)), mode='constant')
# wm1

In [ ]:
# model.load_weights("weights/denoiser")

In [ ]:
def get_optimizer(lr=1e-4):
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
#         optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
        return optimizer
    
def can_save(h):
    has_history = len(h.history["val_loss"]) > 1
#     improved = h.history["val_loss"][0] < h.history["val_loss"][-1]
    not_nan = not tf.math.is_nan(h.history["val_loss"][-1])
    return has_history and not_nan


In [ ]:
class ImageMetric(tf.keras.metrics.Metric):

    def __init__(self, name, **kwargs):
        super(ImageMetric, self).__init__(name=name, **kwargs)
        self.sim = self.add_weight(name='similarity', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        fn = lambda x,y: 0
        if self.name == "ssim_multiscale":
            fn = lambda x,y: tf.image.ssim_multiscale(x,y,255)
        values =fn(y_true,y_pred)

        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, self.dtype)
            sample_weight = tf.broadcast_to(sample_weight, values.shape)
            values = tf.multiply(values, sample_weight)
        self.sim.assign(tf.reduce_mean(values))

    def result(self):
        return self.sim

In [ ]:
class LRMetric(tf.keras.metrics.Metric):

    def __init__(self, optimizer, name="learning_rate", **kwargs):
        super(LRMetric, self).__init__(name, **kwargs)
        self.optimizer = optimizer
        self.lr = self.add_weight(name='lr', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.lr.assign(self.optimizer.lr)

    def result(self):
        return self.lr

## HERE

In [ ]:
from autoencoder_tools.ae_data import div2k_ds

dec_size = DecoderSize.md
batch_size = 4
train_ds = div2k_ds("train", dec_size.value, batch_size=batch_size)
test_ds = div2k_ds("validation", dec_size.value, batch_size=3)


In [ ]:
class RenderCallback(tf.keras.callbacks.Callback):
    def __init__(self, tbutil):
        self.images_renderer = ImageRenderer(3).withTensorboard(tbutil)
        
    def on_train_begin(self, logs=None):
        def ren(m):
            self.images_renderer.render(model=self.model, dataset=test_ds, batch=0)
            return 1
        tf.numpy_function(ren, (1,), Tout=tf.uint8)
        
    def on_epoch_end(self, epoch, logs):
        display.clear_output()
        
        def ren(m):
            self.images_renderer.render(model=self.model, dataset=test_ds, batch=epoch)
            return 1
        tf.numpy_function(ren, (1,), Tout=tf.uint8)


In [ ]:

loss_fn = PeceptualLoss()
mse = tf.keras.losses.MeanSquaredError()
@tf.function
def mse_fn(x,y):
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.float32)
    return tf.math.minimum(mse(x,y), tf.float32.max)

In [ ]:
# tf.keras.backend.clear_session()

In [ ]:
# model.load_weights("weights/denoiser")
# # model.load_weights("weights/denoiser_good")

In [ ]:
# tf.keras.backend.clear_session()

In [ ]:
def check_for_large_memory_vars():
    large_mem_files = list(filter(lambda x:(x[1]/1e6) > 1, [(name, sys.getsizeof(value)) for name, value in globals().items()]))

    if len(large_mem_files) > 0:
        print(f"Found large files: {large_mem_files}. Delete?")
        i = input()
        if "y" in i:
            for (name, size) in large_mem_files:
                del globals()[name]
            print(f"Deleted '{name}''")
check_for_large_memory_vars()

In [ ]:
def save(model, h):
    if can_save(h):
        print("Saving weights")
        model.save_weights("weights/denoiser")

In [ ]:
model = build_small_decoder()

## MSE

In [ ]:
epochs = 30

pl_logdir = f"prepa_autencoder_logs/ae_xs_mse"
pl_tbutil = TensorboardUtil(pl_logdir)

metrics=["mse","accuracy"]

callbacks = [
    RenderCallback(pl_tbutil),
    pl_tbutil.get_callback(),
#     tf.keras.callbacks.TensorBoard(log_dir = "prepa_autencoder_logs/ae_xs_pl",
#                                                  histogram_freq = 1),
    tf.keras.callbacks.EarlyStopping(patience=31),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3)
]

def clipped_mse(x,y):
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.float32) 
    return mse(x,y)

# optimizer = get_optimizer(5e-5)
optimizer = tf.keras.optimizers.experimental.Nadam(learning_rate=5e-5)

model.compile(optimizer=optimizer, loss=clipped_mse, metrics=[*metrics, LRMetric(optimizer)])

h = model.fit(train_ds, epochs=epochs, validation_data=test_ds, callbacks=callbacks)
save(model, h)


In [ ]:
model.save_weights("weights/denoiser")

## YUV MSE

In [ ]:
epochs = 20

yuv_logdir = f"prepa_autencoder_logs/ae_xs_yuv"
yuv_tbutil = TensorboardUtil(yuv_logdir)


callbacks = [
    RenderCallback(yuv_tbutil),
    yuv_tbutil.get_callback(),
    tf.keras.callbacks.EarlyStopping(patience=31),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.7) #, min_delta=1
]

metrics=["mse","accuracy"]

mae = tf.keras.losses.MeanAbsoluteError()
def clipped_mse(x,y):
    x = tf.cast(x , tf.float32)
    y = tf.cast(y , tf.float32) 
    mse_val = mae(tf.image.rgb_to_yuv(x),tf.image.rgb_to_yuv(y))
    return tf.cast(mse_val, tf.float32)

# optimizer = get_optimizer(5e-5)
optimizer = tf.keras.optimizers.experimental.Nadam(learning_rate=1e-5)

model.compile(optimizer=optimizer, loss=clipped_mse, metrics=[*metrics, LRMetric(optimizer)], jit_compile=False)

h = model.fit(train_ds, epochs=epochs, validation_data=test_ds, callbacks=callbacks)
save(model, h)


In [ ]:
# model = build_small_decoder()

## Perceptual Loss

In [ ]:
epochs= 100
pl_logdir = f"prepa_autencoder_logs/ae_xs_pl"
pl_tbutil = TensorboardUtil(pl_logdir)

metrics=["mse","accuracy"]

callbacks = [
    RenderCallback(pl_tbutil),
    pl_tbutil.get_callback(),
#     tf.keras.callbacks.EarlyStopping(patience=30),
    tf.keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=2, factor=0.7)
]

optimizer = get_optimizer(1e-4)
# optimizer = tf.keras.optimizers.experimental.Nadam(learning_rate=1e-4)#, weight_decay=0.001)


model.compile(optimizer=optimizer, loss=loss_fn, metrics=[*metrics, LRMetric(optimizer)])


h = model.fit(train_ds, epochs=epochs,  validation_data=test_ds, callbacks=callbacks)
save(model, h)

## Perceptual Loss (SGD)

In [ ]:
# epochs= 50
# pl_logdir = f"prepa_autencoder_logs/ae_xs_pl_sgd"
# pl_tbutil = TensorboardUtil(pl_logdir)

# metrics=["mse","accuracy"]

# callbacks = [
#     RenderCallback(pl_tbutil),
#     pl_tbutil.get_callback(),
#     tf.keras.callbacks.EarlyStopping(patience=31),
#     tf.keras.callbacks.ReduceLROnPlateau(patience=6)
# ]

# # optimizer = get_optimizer(5e-5)
# optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=1e-6, momentum=0.9, nesterov=True, clipnorm=1.0)


# model.compile(optimizer=optimizer, loss=loss_fn, metrics=[*metrics, LRMetric(optimizer)])


# h = model.fit(train_ds, epochs=epochs,  validation_data=test_ds, callbacks=callbacks)
# save(model, h)

In [ ]:
model = build_small_decoder()
# model.load_weights("weights/denoiser")

In [ ]:
model.save_weights("weights/denoiser")
# model.save_weights("weights/denoiser_good")

In [ ]:
# model.load_weights("weights/denoiser")

## Prepa dataset

In [ ]:


def gen_img(img_path, img_type):
    raw_png = tf.io.read_file(str(img_path), name=img_path)
    return tf.image.decode_png(raw_png, channels=3, name=img_path)

def gen_prepa():
    prepa = [(str(p),'png') for p in pathlib.Path("./prepa").glob('*.png')]
    for img_path, img_type in random.sample(prepa, 10):
        yield gen_img(img_path, img_type), img_path


def test_prepa(autoencoder, examples=10): 
    
    it = iter(gen_prepa())    
    for i in range(examples):
        
        img, img_path = next(it)
        
        
        images = [("Original", img, img_path)]
        
        y_pred = autoencoder.predict(tf.expand_dims(img, axis=0), verbose=0)
        y_pred = tf.cast(y_pred, tf.uint8)[0]
        images.append(("Model", y_pred, img_path))
        
#         gaussian = tfa.image.gaussian_filter2d(tf.expand_dims(img, axis=0))
#         images.append(("gaussian", gaussian[0]))
        
        count = len(images)
        
        fig = plt.figure(figsize=(10*count,20))
        gs = fig.add_gridspec(count, hspace=0)
        axs = gs.subplots()
        for i, (image) in enumerate(images):
            title, im, img_path = image
#             print(img_path)
            axs[i].set_title(title + " " + img_path, y=1.0, pad=-14)
            axs[i].axis('off')
            axs[i].imshow(im)
        
     
        
        

test_prepa(model, 10)

In [ ]:
assert False

In [ ]:
tf.keras.backend.clear_session()
initial_epoch, epochs = em.add_epochs(50)
optimizer = get_optimizer(1e-6)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=[*metrics, LRMetric(optimizer)])


h = model.fit(train_ds, epochs=epochs, initial_epoch=initial_epoch, validation_data=test_ds, callbacks=callbacks)
save(model, h)

In [ ]:
lrt = 0.001
epochs = 20

def decayed_learning_rate(step):
    decay_rate = 0.9
    end_lr = 0.8
    
    return numpy.log(end_lr / 1e-3, step / epochs)

# end_lr = initial_learning_rate * x ^ (step / decay_steps)
# end_lr / initial_learning_rate = x ^ (step / decay_steps)


def get_exploration_schedule(initial_lr, epochs):
    return tf.keras.optimizers.schedules.ExponentialDecay(
        initial_lr,
        decay_steps=epochs,
        decay_rate=0.01,
        staircase=False)

lr_schedule = get_exploration_schedule(lrt, epochs)

for i in range(30):
    lrt = lr_schedule(i)
    print(lrt)

In [ ]:
# tf.keras.backend.clear_session()

In [ ]:
del discriminator

In [ ]:
# tf.keras.backend.clear_session()

lr = 1e-4
model_name = "aenc/xs_v1.1"
dec_size = DecoderSize.md



def get_discriminator(input_shape=(480,720,3)):
    
    def conv(filters, kernel_size=3, strides=1, dilation_rate=(1,1)):
        return layers.Conv2D(filters,
                 kernel_size=3,
                 strides=1,
                 padding="same",
                 use_bias=True,
                dilation_rate=dilation_rate,
                 kernel_regularizer=tf.keras.regularizers.L2(1e-2))
    
    def convblock(filters, x, strides=2, dilation_rate=(1,1)):
        if (strides > 1):
            x = layers.MaxPool2D(strides)(x)
        x = conv(filters, strides=1, dilation_rate=dilation_rate)(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU()(x)
        return x
    
    
    inputs1 = keras.Input(shape=input_shape, name="disc_input")
    input_scaled = tf.keras.layers.Rescaling(1./255)(inputs1)
    
    y = convblock(32, input_scaled, strides=1, dilation_rate=5)
    x = convblock(32, input_scaled, strides=1)
    
    x = tf.add(x,y)
    x = layers.LeakyReLU()(x)
    x = convblock(64, x, strides=2)
    y = convblock(64, x, strides=1, dilation_rate=5)
    x = tf.add(x,y)
    x = layers.LeakyReLU()(x)
    x = convblock(128, x, strides=2)
    x = convblock(512, x, strides=2)
    x = layers.SpatialDropout2D(0.2)(x)
    x = convblock(512, x, strides=2)
    x = layers.SpatialDropout2D(0.2)(x)
    x = convblock(512, x, strides=2)
    x = layers.SpatialDropout2D(0.2)(x)
    x = convblock(256, x, strides=1)
    x = layers.SpatialDropout2D(0.2)(x)
    x = convblock(128, x, strides=1)
    x = layers.SpatialDropout2D(0.2)(x)
    x = convblock(64, x, strides=1)
#     x = convblock(32, x, strides=2)
#     x = down(512, x, strides=2)
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation="leaky_relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1024, activation="leaky_relu")(x)
#     x = layers.Dense(256, activation="leaky_relu")(x)
#     x = layers.Dense(32, activation="leaky_relu", dtype=tf.float16)(x)
    x = layers.Dense(1, activation="softmax")(x)
    return tf.keras.Model(inputs1, x, name="discriminator")


discriminator = get_discriminator((*dec_size.value,3))
discriminator.summary()

In [ ]:
tbutil = TensorboardUtil("prepa_autencoder_logs/encoder_mse")
# dir(tbutil)

In [ ]:
# loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn = PeceptualLoss()
mse = tf.keras.losses.MeanSquaredError()

batch_size = 2
train_ds = div2k_ds("train", dec_size.value).batch(batch_size)
test_ds = div2k_ds("validation", dec_size.value).batch(batch_size)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

def scheduler1(epoch, lr):
    if epoch < 5:
        return 1e-4
    else:
        return 1e-5
sch1 = tf.keras.callbacks.LearningRateScheduler(scheduler1)

images_renderer = ImageRenderer(4).withTensorboard(TensorboardUtil("prepa_autencoder_logs/encoder_mse"))
def on_batch_end(epoch, logs):
    if epoch % 200 == 0:
        images_renderer.render(model=lambda x:encoder.predict(x, verbose=0), datasets=(train_ds, test_ds), batch=epoch)

def on_epoch_end(epoch, logs):
    display.clear_output()
        
render = tf.keras.callbacks.LambdaCallback(on_batch_end=on_batch_end, on_epoch_end=on_epoch_end)
# reduce_callback = tf.keras.callbacks.ReduceLROnPlateau(
#                 monitor="loss",
#                 factor=0.8,
#                 patience=2,
#                 cooldown=1,
#                 min_lr=1e-8)

encoder.compile(optimizer=optimizer, loss=mse, metrics=["accuracy","mse"])

encoder.fit(train_ds, epochs=10, validation_data=test_ds, callbacks=[render, tbutil.get_callback(), sch1])

def scheduler1(epoch, lr):
    if epoch < 10:
        return 1e-4
    if epoch < 20:
        return 1e-5
    if epoch < 30:
        return 1e-6
    else:
        return 1e-7
sch1 = tf.keras.callbacks.LearningRateScheduler(scheduler1)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
encoder.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy","mse"])

encoder.fit(train_ds, epochs=40, validation_data=test_ds, callbacks=[render, tbutil.get_callback(), sch1])


In [ ]:
encoder.save_weights("weights/encoder_mse")

In [ ]:
encoder.load_weights("weights/encoder_mse")


In [ ]:
discriminator = get_discriminator((*dec_size.value,3))

In [ ]:
discriminator.load_weights("weights/discriminator_mse")

In [ ]:
model.load_weights("weights/denoiser")
encoder = model

In [ ]:
mse = tf.keras.losses.MeanSquaredError()
loss_fn = tf.keras.losses.BinaryCrossentropy()

batch_size = 2
epochs = 30
epoch_start_enc_train = 0



is_enc_training = True

def get_optimizer(lr=1e-4):
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    optimizer = mixed_precision.LossScaleOptimizer(optimizer)
    return optimizer

opt_enc =get_optimizer(1e-4)
opt_disc =get_optimizer(1e-4)

images_renderer = ImageRenderer(4).withTensorboard(TensorboardUtil("prepa_autencoder_logs/discriminator"))
print(f"Output size: {dec_size.value}")
train_ds = div2k_ds("train", dec_size.value).batch(batch_size)
test_ds = div2k_ds("validation", dec_size.value).batch(batch_size)

train_writer = tf.summary.create_file_writer("prepa_autencoder_logs/discriminator/train")
val_writer = tf.summary.create_file_writer("prepa_autencoder_logs/discriminator/test")

int_spec = tf.TensorSpec(shape=[], dtype=tf.int32)
image_spec1 = tf.TensorSpec(shape=[None,480, 720, 3], dtype=tf.float16)
image_spec = tf.TensorSpec(shape=[None,*dec_size.value,3], dtype=tf.float16)
int_spec = tf.TensorSpec(shape=[], dtype=tf.int32)
bool_spec = tf.TensorSpec(shape=[], dtype=tf.bool)
disc_output_spec = tf.TensorSpec(shape=[None,1], dtype=tf.float16, name="disc_output")

# ploss = LossNetwork()

def resize(x):
    return tf.image.resize(
            x,
            (dec_size.value[0], dec_size.value[1]),
            preserve_aspect_ratio=False,
            antialias=False,
            method="nearest",
            name=None)

# @tf.function(input_signature=(disc_output_spec,disc_output_spec))
def disc_loss_fn(y_pred,y_true):
    l1 = loss_fn(tf.ones_like(y_true),y_true)
    l2 = loss_fn(tf.zeros_like(y_pred),y_pred)
    return tf.cast(l1+l2, tf.float32)

# @tf.function(input_signature=(disc_output_spec,disc_output_spec))
def enc_loss_fn(y_pred):
    l1 = loss_fn(tf.ones_like(y_pred), y_pred)
    return tf.cast(l1, tf.float32)



def calculate_val():
    enc_loss = 0
    disc_loss = 0
    steps = 0
    for step, (x, y) in enumerate(test_ds):
        steps+=1

        x = encoder(x, training=False)
    
#         z = resize(y)
        y_pred = discriminator(x, training=False)
        y_true = discriminator(y, training=False)
#         noise = discriminator(z, training=False)
        
        dloss = disc_loss_fn(y_pred,y_true)
        eloss = enc_loss_fn(y_pred)
#         eloss = mse(x, y)

        enc_loss += eloss
        disc_loss += dloss
    return enc_loss/steps, disc_loss/steps
        
def update_gradients(model, tape, optimizer, scaled_loss):
    scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
    gradients = optimizer.get_unscaled_gradients(scaled_gradients)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

@tf.function(input_signature=(int_spec, int_spec ,image_spec1, image_spec, bool_spec))
def train_step(epoch, step, x, y, is_enc_training):
    # Open a GradientTape to record the operations run
    # during the forward pass, which enables auto-differentiation.
    tape_enc = tf.GradientTape()
    tape_disc = tf.GradientTape()
    with tape_enc:
#         is_enc_training = epoch >= epoch_start_enc_train
        x = encoder(x, training=is_enc_training)
        
   
    with tape_disc, tape_enc:
#         z = resize(y)
        y_pred = discriminator(x, training=True)
        y_true = discriminator(y, training=True)
#         noise = discriminator(z, training=True)
#         print(f"Sizes: y_pred={y_pred.shape} y_true={y_true.shape}")
        
        dloss = disc_loss_fn(y_pred,y_true)
        dscaled_loss = opt_disc.get_scaled_loss(dloss)

        eloss = tf.cast(0.0, tf.float32)
        escaled_loss = tf.cast(0.0, tf.float32)
        mse_error = tf.cast(0.0, tf.float32)
        if is_enc_training:
            mse_error =  mse(tf.cast(x, tf.float32),tf.cast(y, tf.float32))
            eloss = enc_loss_fn(y_pred)
    #             eloss = mse(x, y)
            escaled_loss = opt_enc.get_scaled_loss(eloss * mse_error)

        
        
#         loss_value = calculate_loss(x_batch_train,y_batch_train, training=True)

    if is_enc_training:
        update_gradients(encoder, tape_enc, opt_enc, escaled_loss)
    update_gradients(discriminator, tape_disc, opt_disc, dscaled_loss)
    return eloss, dloss, mse_error


images_renderer.render(model=lambda x:encoder.predict(x, verbose=0), datasets=(train_ds, test_ds), batch=0)
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    
    epoch_enc_loss = 0
    epoch_disc_loss = 0
    epoch_mse = 0
    steps = 0
    # Iterate over the batches of the dataset.
    progress = iter(trange(len(train_ds)))
    for step, (x_batch_train, y_batch_train) in enumerate(train_ds):
        steps += 1
        encoder_loss, disc_loss, mse_error = train_step(epoch, step, x_batch_train, y_batch_train, is_enc_training)
        epoch_enc_loss += encoder_loss
        epoch_disc_loss += disc_loss
        epoch_mse += mse_error
        next(progress)
        # Log every 200 batches.
        
        if step % 20 == 0:
            print(f"encoder_loss={encoder_loss}, discriminator_loss={disc_loss} mse={mse_error}")
#             print("Seen so far: %s samples" % ((step + 1) * batch_size))
    val_enc_loss, val_disc_loss = calculate_val()
    display.clear_output()
    with train_writer.as_default():
        tf.summary.scalar("enc_loss", epoch_enc_loss/steps, step=epoch)
        tf.summary.scalar("disc_loss", epoch_disc_loss/steps, step=epoch)
        tf.summary.scalar("mse", epoch_mse/steps, step=epoch)
    with val_writer.as_default():
        tf.summary.scalar("enc_loss", val_enc_loss, step=epoch)
        tf.summary.scalar("disc_loss", val_disc_loss, step=epoch)
    images_renderer.render(model=lambda x:encoder.predict(x, verbose=0), datasets=(train_ds, test_ds), batch=epoch)
    
    print(f"Validation enc_loss = {val_enc_loss} disc_loss = {val_disc_loss}")


In [ ]:
discriminator.save_weights("weights/discriminator_mse")

In [ ]:
# encoder.save_weights("weights/encoder_mse")

In [ ]:
test_prepa(model, 10)

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = get_optimizer(1e-5)
def combined_loss(x,y):
    discriminator(x, training=False)
    return (loss_fn(x,y)) * mse_fn(x,y)

model.compile(optimizer=get_optimizer(lr=1e-5), loss=combined_loss, metrics=["accuracy","mse"])


model.fit(train_ds, epochs=50, validation_data=test_ds, callbacks=[render, tbutil.get_callback()])
